In [1]:
!pip install -U -q transformers==4.39.3
!pip install -U -q accelerate==0.28.0
!pip install -U -q datasets==2.18.0
!pip install -U -q peft==0.10.0
!pip install -U -q bitsandbytes==0.43.1
!pip install -U -q trl==0.8.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.6.1 requires fsspec==2024.6.1, but you have fsspec 2024.2.0 which is incompatible.
   ━━━━━━━━

In [2]:
import warnings

warnings.filterwarnings("ignore")

In [3]:
import os
from huggingface_hub import login
from google.colab import userdata

hf_token = userdata.get('HF_TOKEN')
wandb_api_key = userdata.get('wandb')

login(token=hf_token)

os.environ["WANDB_API_KEY"] = wandb_api_key
os.environ["WANDB_PROJECT"] = "Fine-tuning openELM-270m with ultrafeedback"
os.environ["WANDB_NAME"] = "ft-openelm-270m-ultrafeedback"
os.environ["MODEL_NAME"] = "apple/OpenELM-270M"
os.environ["TOKENIZER_NAME"] = "meta-llama/Llama-2-7b-hf"
os.environ["DATASET"] = "HuggingFaceH4/ultrafeedback_binarized"

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
from transformers import AutoTokenizer

tokenizer=AutoTokenizer.from_pretrained(
    os.getenv("TOKENIZER_NAME"),
    add_eos_token=True,
    use_fast=True)

tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="left"

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [5]:
from datasets import load_dataset

ds=load_dataset(os.getenv("DATASET"), split=["train_prefs","test_prefs"])
ds

Generating train_prefs split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating train_sft split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating test_prefs split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test_sft split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/1000 [00:00<?, ? examples/s]

[Dataset({
     features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
     num_rows: 61135
 }),
 Dataset({
     features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
     num_rows: 2000
 })]

In [6]:
train_ds=ds[0].shuffle(seed=42).select(range(3000))
eval_ds=ds[1].shuffle(seed=42).select(range(1000))

print(train_ds)
print(eval_ds)

Dataset({
    features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
    num_rows: 3000
})
Dataset({
    features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
    num_rows: 1000
})


In [7]:
import torch, multiprocessing

def preprocess(x):
    x["chosen"]=tokenizer.apply_chat_template(x["chosen"], tokenize=False)
    x["rejected"]=tokenizer.apply_chat_template(x["rejected"], tokenize=False)
    return x

train_ds=train_ds.map(preprocess, num_proc=multiprocessing.cpu_count(), load_from_cache_file=False)
eval_ds=eval_ds.map(preprocess, num_proc=multiprocessing.cpu_count(), load_from_cache_file=False)

Map (num_proc=8):   0%|          | 0/3000 [00:00<?, ? examples/s]


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.


No chat template is defined for this tokenizer - using

Map (num_proc=8):   0%|          | 0/1000 [00:00<?, ? examples/s]


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.


No chat template is defined for this tokenizer - using

In [8]:
from transformers import AutoModelForCausalLM
model=AutoModelForCausalLM.from_pretrained(
    os.getenv("MODEL_NAME"),
    torch_dtype=torch.float16,
#     device_map={"": 0},
    device_map="cuda",
    trust_remote_code=True
)

model.gradient_checkpointing_enable()
model.device

config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

configuration_openelm.py:   0%|          | 0.00/14.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/apple/OpenELM-270M:
- configuration_openelm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_openelm.py:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/apple/OpenELM-270M:
- modeling_openelm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

device(type='cuda', index=0)

In [11]:
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.8 MB/s eta 0:00:00


In [12]:
from trl import ORPOTrainer, ORPOConfig

orpo_config=ORPOConfig(
    output_dir=os.getenv("WANDB_NAME"),
    evaluation_strategy="steps",
    do_eval=True,
    optim="adamw_8bit",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=8,
    log_level="debug",
    logging_steps=100,
    learning_rate=8e-6,
    eval_steps=100,
    save_steps=100,
    save_strategy="epoch",
    num_train_epochs=1,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    beta=0.1, # beta is ORPO's lambda
    max_length=1024,
    report_to="wandb",
    run_name=os.getenv('WANDB_NAME')
)

trainer = ORPOTrainer(
        model=model,
        train_dataset=train_ds,
        eval_dataset=eval_ds,
        args=orpo_config,
        tokenizer=tokenizer,
)

trainer.train()

PyTorch: setting up devices
You have loaded a model on multiple GPUs. `is_model_parallel` attribute will be force-set to `True` to avoid any unexpected behavior such as device placement mismatching.
Currently training with a batch size of: 8
***** Running training *****
  Num examples = 3,000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 187
  Number of trainable parameters = 271,527,168
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: sosamaali (midconstruct). Use `wandb login --relogin` to force relogin


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Runtime,Samples Per Second,Steps Per Second,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen,Nll Loss,Log Odds Ratio,Log Odds Chosen
100,1.759400,1.645508,110.929100,9.015000,1.127000,-0.199341,-0.202881,0.505000,0.003475,-2.027344,-1.994141,-5.738281,-6.109375,1.574219,-0.703727,0.044539


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to ft-openelm-270m-ultrafeedback/checkpoint-187
Configuration saved in ft-openelm-270m-ultrafeedback/checkpoint-187/config.json
Configuration saved in ft-openelm-270m-ultrafeedback/checkpoint-187/generation_config.json
Model weights saved in ft-openelm-270m-ultrafeedback/checkpoint-187/model.safetensors
tokenizer config file saved in ft-openelm-270m-ultrafeedback/checkpoint-187/tokenizer_config.json
Special tokens file saved in ft-openelm-270m-ultrafeedback/checkpoint-187/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=187, training_loss=1.7277675363469251, metrics={'train_runtime': 1391.4348, 'train_samples_per_second': 2.156, 'train_steps_per_second': 0.134, 'total_flos': 0.0, 'train_loss': 1.7277675363469251, 'epoch': 1.0})

In [13]:
kwargs={
    'model_name': os.getenv("WANDB_NAME"),
    'finetuned_from': os.getenv('MODEL_NAME'),
#     'tasks': 'Text-Generation',
#     'dataset_tags':'',
    'dataset': os.getenv("DATASET")
}

tokenizer.push_to_hub(os.getenv("WANDB_NAME"))
trainer.push_to_hub(**kwargs)

tokenizer config file saved in ft-openelm-270m-ultrafeedback/tokenizer_config.json
Special tokens file saved in ft-openelm-270m-ultrafeedback/special_tokens_map.json
Uploading the following files to OsamaAliMid/ft-openelm-270m-ultrafeedback: special_tokens_map.json,tokenizer.json,README.md,tokenizer.model,tokenizer_config.json


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

Saving model checkpoint to ft-openelm-270m-ultrafeedback
Configuration saved in ft-openelm-270m-ultrafeedback/config.json
Configuration saved in ft-openelm-270m-ultrafeedback/generation_config.json
Model weights saved in ft-openelm-270m-ultrafeedback/model.safetensors
tokenizer config file saved in ft-openelm-270m-ultrafeedback/tokenizer_config.json
Special tokens file saved in ft-openelm-270m-ultrafeedback/special_tokens_map.json


model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/OsamaAliMid/ft-openelm-270m-ultrafeedback/commit/54b90c3384d22fc0ef642bd15afaf55a8e11324f', commit_message='End of training', commit_description='', oid='54b90c3384d22fc0ef642bd15afaf55a8e11324f', pr_url=None, pr_revision=None, pr_num=None)

In [14]:
model=AutoModelForCausalLM.from_pretrained(
    os.getenv("WANDB_NAME"),
    torch_dtype=torch.float16,
    device_map="cuda",
    trust_remote_code=True
)

chat=[
    [{"role":"user","content":"How is vanilla cultivated?"}],
    [{"role": "user", "content": "How much money do I have if I have one dollar?"}],
    [{"role": "user", "content": "Where is Berlin?"}],
    [{"role": "user", "content": "Give me a list of 5 European countries."}],
    [{"role": "user", "content": "What is AI?"}],
    [{"role": "user", "content": "What can you do right? Exactly?"}]
]


for c in chat:
    p=tokenizer.apply_chat_template(c, tokenize=False)
    inputs = tokenizer(p, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, do_sample=True, pad_token_id=tokenizer.eos_token_id, top_p=0.9, max_new_tokens=150)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(result)

loading configuration file ft-openelm-270m-ultrafeedback/config.json
loading configuration file ft-openelm-270m-ultrafeedback/config.json
Model config OpenELMConfig {
  "_name_or_path": "ft-openelm-270m-ultrafeedback",
  "activation_fn_name": "swish",
  "architectures": [
    "OpenELMForCausalLM"
  ],
  "auto_map": {
    "AutoConfig": "apple/OpenELM-270M--configuration_openelm.OpenELMConfig",
    "AutoModelForCausalLM": "apple/OpenELM-270M--modeling_openelm.OpenELMForCausalLM"
  },
  "bos_token_id": 1,
  "eos_token_id": 2,
  "ffn_dim_divisor": 256,
  "ffn_multipliers": [
    0.5,
    0.73,
    0.97,
    1.2,
    1.43,
    1.67,
    1.9,
    2.13,
    2.37,
    2.6,
    2.83,
    3.07,
    3.3,
    3.53,
    3.77,
    4.0
  ],
  "ffn_with_glu": true,
  "head_dim": 64,
  "initializer_range": 0.02,
  "max_context_length": 2048,
  "model_dim": 1280,
  "model_type": "openelm",
  "normalization_layer_name": "rms_norm",
  "normalize_qk_projections": true,
  "num_gqa_groups": 4,
  "num_kv_head

[INST] How is vanilla cultivated? [/INST].
[INST] Why doesn't it have an "Evacuate the Hole" function? It seems like it's the most used item. What is it used for? [/INST] We're going to use this to illustrate the fact that vanilla cultivated items don't have a "Evacuate the Hole" function.
[INST] What is vanilla cultivated in the "Cultivation" item? [/INST] I assume that "Cultivated" indicates that the item is a plant with no physical growth mechanisms, but it is a fruit. [/INST]
[INST] What is the type of vanilla tree? [/INST
[INST] How much money do I have if I have one dollar? [/INST]


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


[INST] Where is Berlin? [/INST]>
<!--->
I'm not sure why the "Institution" is not available, but I guess it has to do with how the website is currently configured to display the first three letter words:
[/INST]
<!--->
Hi,
Thanks for the report, I've just added the institute of the Institute of Technology to my list of places to visit. It seems to be the biggest institute in Berlin, with many buildings, a good mix of architecture, engineering and design.
There are also some things to note -
1. it's really weird to say "Schloss Wilhelm IV", I'm not sure it was a school or university, it seems like a
[INST] Give me a list of 5 European countries. [/INST] [/INST]


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


[INST] What is AI? [/INST]

[Inst] What is AI?

[INST] What is AI? What Is AI? The question of whether or not you have AI is a question that no one really answers. It is a big debate that has not yet been settled. However, I believe that what we can do today is at least have a baseline understanding of what AI is, so that we can answer this question in the future.

The basics of AI

AI has the following characteristics:

A large variety of machines with multiple tasks

Reliability

Movement ability

Increased cognitive power

The best approach to AI is to ask:


[INST] What can you do right? Exactly? [/INST]
